# 사전준비

In [1]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.formula.api import logit

### 안전도 데이터

In [2]:
safety = pd.read_excel('data/safety.xlsx')
safety.head()

,risk,fuel,aspiration,doors,body,wheels,engine_location,wheel_base,length,width,height,curb_weight,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg
0,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,3.19,3.4,10.0,102,5500,24,30
1,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,3.19,3.4,8.0,115,5500,18,22
2,1,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,3.19,3.4,8.5,110,5500,19,25
3,1,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,3.13,3.4,8.3,140,5500,17,20
4,2,gas,std,two,sedan,rwd,front,101.2,176.8,64.8,54.3,2395,3.50,2.8,8.8,101,5800,23,29


### 고객이탈 데이터

In [3]:
churn = pd.read_excel('data/churn.xlsx')
churn.head()

,has_churned,time_since_first_purchase,time_since_last_purchase
0,0,-1.089221,-0.721322
1,0,1.182983,3.634435
2,0,-0.846156,-0.427582
3,0,0.086942,-0.535672
4,0,-1.166642,-0.672640


# 분류 분석
종속변수(y)가 범주형 변수인 지도 학습<br>
예시
- 병원에 도착한 환자의 응급/비응급
- 고객의 이탈/유지
- 제품의 고장/정상
- 금융거래의 사기/정상

## 대표적인 분류 분석의 종류
- 로지스틱 회귀분석
- 판별 분석
- K-최근접 이웃
- 서포트 벡터 머신

## 분류의 종류
- 이항 분류(binary classification): 2개의 범주로 분류
- 다항 분류(multinomial/multiclass classification): 3개 이상의 범주로 분류

## 이항 분류에서 양성과 음성
- 양성(positive): 탐지하고자 하는 대상이 있음. (예: 암, 불량, 화재 등)
 - 긍정적이라는 의미의 양성 benign이 아님
- 음성(negative): 탐지하고자 하는 것이 없음.

---
# 로지스틱 회귀분석
선형 모형에 로지스틱 함수 σ를 적용
```python
p = σ(wx + b)
```
양성/음성 중 양성일 확률을 예측

## 로지스틱 함수
![로지스틱 함수](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/320px-Logistic-curve.svg.png)
```python
σ(z) = 1 / (1 + exp(z)
```
- z가 증가할수록 출력이 1에 가까워짐 (확률 증가)
- z가 감소할수록 출력이 0에 가까워짐 (확률 감소)
```python
z = wx + b
```
- 로지스틱 함수는 y=xw+b의 확률을 구함
 - 확률 p = f(y) = f(wx+b)

---
## Python 로지스틱 회귀분석
1. import
```python
from statsmodels.formula.api import logit
```
2. 분석
```python
res = logit('y ~ x', df).fit()
res.summary()
```

- `y` 는 0, 1로 더미코딩되어 있어야 함.
- 기울기가 +면 (1에 해당하는 범주의) 확률이 증가
 - 기울기가 -면 확률이 감소

### 더미코딩

In [4]:
safety['fuel_gas'] = safety.fuel.map({'gas': 0, 'diesel': 1})

### 분석

In [5]:
res = logit('fuel_gas ~ city_mpg + peak_rpm', data=safety).fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.165082
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               fuel_gas   No. Observations:                  159
Model:                          Logit   Df Residuals:                      156
Method:                           MLE   Df Model:                            2
Date:                Fri, 09 Oct 2020   Pseudo R-squ.:                  0.4717
Time:                        19:53:02   Log-Likelihood:                -26.248
converged:                       True   LL-Null:                       -49.682
Covariance Type:            nonrobust   LLR p-value:                 6.650e-11
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     20.8568      5.527      3.774      0.000      10.024      31.689
city_mpg       0.1560      0.056      2.776      0.006       0.046       0.266
peak_rpm      -0.0058      0.001     -4.380      0.000      -0.008      -0.003
==============================================================================
"""

### 해석
- `-0.1560`: city_mpg가 올라갈수록 gas일 확률이 떨어짐
 - diesel의 확률은 올라감
- `0.0058`: peak_rpm이 올라갈수록 gas의 확률이 올라감
 - diesel의 확률은 떨어짐

### 적합도 지수
로지스틱 회귀분석에서는 aic와 bic 값이 summary에 나오지 않으므로, 별도로 변수에 저장해서 출력해줘야 함
- 수정 R제곱은 없음

`AIC`
```python
res.aic
```

`BIC`
```python
res.bic
```

`McFadden의 유사 R제곱`
```python
res.prsquared
```

In [6]:
res.aic

58.49617328968348

In [7]:
res.bic

67.70288589634418

---
## 로지스틱 회귀분석 실습
고객이탈 데이터를 다운로드 받아 `has_churned` 변수를 예측하는 로지스틱 회귀분석을 수행해보세요.<br>
독립변수에는 `time_since_first_purchase`와 `time_since_last_purchase`를 넣어보세요.
- `has_churned`: 이탈한 고객(0=유지, 1=이탈)
- `time_since_first_purchase`: 가입기간
- `time_since_last_purchase`: 마지막 구매후 기간

In [8]:
churn_log = logit('has_churned ~ time_since_first_purchase + time_since_last_purchase', data=churn).fit()
churn_log.summary()

Optimization terminated successfully.
         Current function value: 0.651175
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            has_churned   No. Observations:                  400
Model:                          Logit   Df Residuals:                      397
Method:                           MLE   Df Model:                            2
Date:                Fri, 09 Oct 2020   Pseudo R-squ.:                 0.06055
Time:                        19:53:02   Log-Likelihood:                -260.47
converged:                       True   LL-Null:                       -277.26
Covariance Type:            nonrobust   LLR p-value:                 5.113e-08
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -0.1168      0.108     -1.082      0.279      -0.328       0.095
time_since_first_purchase    -0.6478      0.139     -4.655      0.000      -0.920      -0.375
time_since_last_purchase      0.5122      0.113      4.514      0.000       0.290       0.735
=============================================================================================
"""

### `time_since_first_purchase`의 기울기

In [9]:
churn_log.params['time_since_first_purchase']

-0.6477753811967569

### `time_since_last_purchase`의 기울기

In [10]:
churn_log.params['time_since_last_purchase']

0.5121576876062008

### 가입기간이 짧고, 마지막 구매 후 시간이 오래 지난 고객이 이탈할 확률이 높다.
- 가입기간이 길수록 이탈 확률이 떨어진다(-0.6478)
 - 가입기간이 짧을수록 이탈 확률이 올라간다
- 마지막 구매후 시간이 오래 지날수록 이탈 확률이 올라간다(0.5122)

### 어떤 모형이 고객 이탈을 가장 잘 예측할까?
1. has_churned ~ time_since_first_purchase + time_since_last_purchase
2. has_churned ~ time_since_first_purchase
3. has_churned ~ time_since_last_purchase

순으로 잘 예측한다.

In [11]:
churn_log.aic, churn_log.bic

(526.9399128658538, 538.9143065071777)

In [12]:
first_log = logit('has_churned ~ time_since_first_purchase', data=churn).fit()
first_log.aic, first_log.bic

Optimization terminated successfully.
         Current function value: 0.679663
         Iterations 4


(547.7307045461821, 555.713633640398)

In [13]:
last_log = logit('has_churned ~ time_since_last_purchase', data=churn).fit()
last_log.aic, last_log.bic

Optimization terminated successfully.
         Current function value: 0.683000
         Iterations 4


(550.400290547221, 558.3832196414369)